In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

In [ ]:
dados = pd.read_csv("Dmoz-sports.csv")

In [ ]:
# Create PyTorch datasets for model training
train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)
test_dataset = BertDataset(test_encodings, test_labels)

In [ ]:
# Load model
num_labels = len(label_encoder.classes_)
model = get_model(num_labels)

# Train model
trainer = train_model(model, train_dataset, val_dataset, tokenizer)

# Evaluate model
accuracy, f1_micro, f1_macro, conf_matrix = evaluate_model(trainer, test_dataset, test_labels, label_encoder)

In [ ]:
# Initialize the BERT tokenizer
tokenizer = get_tokenizer()

# Tokenize the text data for each split
# Tokenize training data and get analysis samples
train_encodings, train_sample_text, train_tokens, train_token_ids = tokenize_texts(train_texts, tokenizer)

# Tokenize validation and test data (only need encodings)
val_encodings = tokenize_texts(val_texts, tokenizer)[0]
test_encodings = tokenize_texts(test_texts, tokenizer)[0]